In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
import language_tool_python
##  StratifiedKFold、cross_val_scoreをインポート
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score


In [20]:
test = pd.read_csv('../input/test_essays.csv')
sub = pd.read_csv('../input/sample_submission.csv')
external_train = pd.read_csv("../input/train_essays.csv")
external_train.rename(columns={"generated": "label"}, inplace=True)

train = pd.read_csv("../input/train_v2_drcat_02.csv")
train = pd.concat([train, external_train])

train = train.drop_duplicates(subset=["text"])

train.reset_index(drop=True)



,text,label,prompt_name,source,RDizzl3_seven,id,prompt_id
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False,NaN,NaN
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False,NaN,NaN
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False,NaN,NaN
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False,NaN,NaN
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False,NaN,NaN
...,...,...,...,...,...,...,...
44863,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True,NaN,NaN
44864,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True,NaN,NaN
44865,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True,NaN,NaN
44866,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True,NaN,NaN


In [21]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB

# TF-IDFベクトル化器を作成
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

test_x = tfidf.fit_transform(test["text"])
train_x = tfidf.transform(train["text"])

lr = LogisticRegression()
clf = MultinomialNB(alpha=0.02)
sgd_model = SGDClassifier(max_iter=5000, tol=1e-3, loss="modified_huber")   
sgd_model2 = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber", class_weight="balanced") 
sgd_model3 = SGDClassifier(max_iter=10000, tol=5e-4, loss="modified_huber", early_stopping=True)

ensemble = VotingClassifier(
    estimators=[
        ("lr", lr),
        ("mnb", clf),
        ("sgd", sgd_model),
        ("sgd2", sgd_model2),
        ("sgd3", sgd_model3),
    ],
    voting="soft",
)
ensemble.fit(train_x, train.label)


VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('mnb', MultinomialNB(alpha=0.02)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=5000)),
                             ('sgd2',
                              SGDClassifier(class_weight='balanced',
                                            loss='modified_huber',
                                            max_iter=8000, tol=0.0001)),
                             ('sgd3',
                              SGDClassifier(early_stopping=True,
                                            loss='modified_huber',
                                            max_iter=10000, tol=0.0005))],
                 voting='soft')

In [22]:
test["generated"] = ensemble.predict_proba(test_x)[:, 1]


In [23]:
#kaggleの提出用ファイルを作成
test[["id", "generated"]].to_csv("submission.csv", index=False)
